### MR3522 Lab 4: An Introduction to Microwave Data from SSMI/S

#### Description: Plot a sweep and swath of SSMI/S data at various channels over a West Pacific typhoon.

#### Import modules.

In [23]:
import numpy as np
from glob import glob
import netCDF4 as nc4
from matplotlib import pyplot as plt, cm
import ipywidgets as widgets
import boto3
import os

%matplotlib notebook

#### Point to directory containing data.

In [18]:
s3 = boto3.resource('s3')
s3.Bucket('nps-mr3522').download_file('Lab4/RSS_SSMIS_FCDR_V07R00_F17_D20180901_S0841_E1034_R61016.nc', 'data.nc')

#### Open file and read in data.

In [25]:
Tb = {}

#%% Load the data
ncid = nc4.Dataset('data.nc','r')
lat = ncid['latitude_lores'][:,:]
lon = ncid['longitude_lores'][:,:]
lat92 = ncid['latitude_hires'][:,:]
lon92 = ncid['longitude_hires'][:,:]
Tb['H19'] = ncid['fcdr_brightness_temperature_19H'][:,:]
Tb['V19'] = ncid['fcdr_brightness_temperature_19V'][:,:] 
Tb['V22'] = ncid['fcdr_brightness_temperature_22V'][:,:]
Tb['H37'] = ncid['fcdr_brightness_temperature_37H'][:,:]
Tb['V37'] = ncid['fcdr_brightness_temperature_37V'][:,:]
Tb['H92'] = ncid['fcdr_brightness_temperature_92H'][:,:]
Tb['V92'] = ncid['fcdr_brightness_temperature_92V'][:,:]
os.remove('data.nc')

#### Select the specific area we want. For this file, we're looking over Typhoon Jebi specifically, which was near 20N, 138E.

The first line below plots a subset of the data plotted by the second. Run the code with the first line uncommented first, then when the lab directs you to, comment out the first line below and uncomment the second line to plot a larger swath of data.

In [20]:
#I = np.arange(1100,1102) #This covers just two sweeps.
I = np.arange(1000,1200) #This covers the part of the ascending node that covered the typhoon.

#### Function for plotting.

In [21]:
def InteractivePlot(band):

  plt.close('all')
    
  Tbuse = Tb[band]
    
  fig,ax = plt.subplots(1,1,figsize=(5,7.5))
  
  if band == 'H92' or band == 'V92':
    Xflat, Yflat, Zflat = lon92[I,:].flatten(), lat92[I,:].flatten(), Tbuse[I,:].flatten()
    cf = ax.contourf(lon92[I,:],lat92[I,:],Tbuse[I,:],np.arange(150,290,10),extend="both",cmap=cm.get_cmap('plasma'))

  else:
    Xflat, Yflat, Zflat = lon[I,:].flatten(), lat[I,:].flatten(), Tbuse[I,:].flatten()
    cf = ax.contourf(lon[I,:],lat[I,:],Tbuse[I,:],np.arange(150,290,10),extend="both",cmap=cm.get_cmap('plasma'))

  cb = fig.colorbar(cf, ax=ax)
  cb.set_label('Brightness Temperature (K)')
  ax.set_xlabel('Longitude')
  ax.set_ylabel('Latitude')
    
  # Function for getting values to show up when hovering over plot.
  def fmt(x, y):
    # get closest point with known data
    dist = np.linalg.norm(np.vstack([Xflat - x, Yflat - y]), axis=0)
    idx = np.argmin(dist)
    z = Zflat[idx]
    return 'Longitude={x:.2f}  Latitude={y:.2f}  Tb={z:.2f}'.format(x=x, y=y, z=z)
    
  ax.format_coord = fmt

In [22]:
bands = ['H19','V19','V22','H37','V37','H92','V92']
varnames = widgets.Dropdown(options=bands,description='SSMI/S Channels')
widgets.interactive(InteractivePlot,band=varnames)

interactive(children=(Dropdown(description='SSMI/S Channels', options=('H19', 'V19', 'V22', 'H37', 'V37', 'H92…